# 데이터 변환하기

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import mean_squared_error

In [3]:
# 더미데이터를 구해서 연습해보겠습니다
libraries = pd.read_csv('./ratings.csv')
libraries.columns = ['account_id', 'book_id', 'stars', 'read_status']

In [4]:
# read_status에 이상한 숫자들이 들어있는데, 추천 계산 시에는 들어가지 않으므로 상관없습니다 1 
libraries.head(3)

,account_id,book_id,stars,read_status
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [5]:
# read_status에 이상한 숫자들이 들어있는데, 추천 계산 시에는 들어가지 않으므로 상관없습니다 2
col = ['account_id', 'book_id', 'stars']
libraries = libraries[col]

# 행렬분해로 평점 예측하기

In [6]:
# DB에는 사용자별, 책별 평점을 저장되어 있습니다. 
# 추천 알고리즘으로 활용하기 위해서 pivot table 형태로 변환해줍니다
rating_matrix = libraries.pivot_table('stars', index='account_id', columns='book_id')
rating_matrix = rating_matrix.fillna(0)
R = rating_matrix.values

In [7]:
# parameters
# 속도를 개선하려면 K는 작게, steps는 작게 하면 좋습니다
K = 50          # 잠재요인 차원
steps = 100     # SGD 횟수
learning_rate = 0.01
r_lambda = 0.01 # lasso 규제 적용

In [17]:
R.head(3)

book_id,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
account_id,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# 사용자 600명, 책 10000여권이 있을 때를 가정한 데이터입니다
num_users, num_items = R.shape
print(num_users, num_items)

610 9724


In [9]:
# 사용자 평점 데이터 R에서 필요한 데이터만 추출한 후 이 행렬을 행렬 P, Q로 분해할 것입니다.
# P, Q의 initial value를 랜덤하게 생성해줍니다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [10]:
prev_rmse = 10000
break_count = 0

In [11]:
# P, Q로 분해하기 전에 사용자 평점 데이터 R에서 데이터를 추출하겠습니다.
# 사용자 평점 데이터 R에서 non zero인 아이들만 추려내는 과정입니다.
non_zeros = [ (i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]


In [12]:
# 알고리즘을 평가하기 위한 기준(metric)에는 rmse, mse, entropy 등등이 있습니다.
# 데이터가 연속형(continuous)이므로 metric으로 mean_squared_error를 활용했습니다.
def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero  in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    return rmse

In [49]:
# 역전파를 응용해 P, Q를 업데이트하겠습니다
# 속도를 개선하려면 K는 작게, steps는 작게 하면 좋습니다
for step in range(steps):
    for i,j,r in non_zeros:
        eij = r -np.dot(P[i,:], Q[j,:].T)
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j,:] - r_lambda * P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i,:] - r_lambda * Q[j,:])

    rmse = get_rmse(R,P,Q,non_zeros)
    if (step % 10) == 0:
        print(step, rmse)


0 1.1789273867918946
10 0.7161027396357864
20 0.49450426798225194
30 0.3663214185165719
40 0.29579348823342144
50 0.2548134366576569
60 0.22922841294959573
70 0.2120360690058328
80 0.19974407807678873
90 0.1905251377037576
100 0.18335311081297764
110 0.17761182652152743
120 0.17291053222369204
130 0.16899021664636454
140 0.16567265615004795
150 0.1628311543583286
160 0.1603730173770825
170 0.15822868699649525
180 0.1563447922624721
190 0.154679578953332


# 예측 후 추천 아이템 추출하기

In [50]:
# 이것이 우리가 예측한 평점입니다
pred_matrix = np.dot(P, Q.T)

In [53]:
# AI를 활용하기 좋은 데이터 타입인 dataframe 자료형으로 변환하겠습니다
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=rating_matrix.index, columns=rating_matrix.columns)

In [68]:
# account_id가 9인 유저가 좋아할 만한 책을 10개 추출하겠습니다
# 이미 본 책은 예외로 두겠습니다 (already_seen)
account_id = 9
top_n = 10

In [73]:
user_rating = rating_matrix.loc[account_id, :]
already_seen = user_rating[user_rating>0].index.tolist()
books_list = rating_matrix.columns.tolist()
unseen_list = [book for book in books_list if book not in already_seen]

In [74]:
recom_books = ratings_pred_matrix.loc[account_id, unseen_list].sort_values(ascending=False)[:top_n]

In [79]:
ans = list(recom_books.index)
print(ans)

[1291, 2064, 7153, 1230, 63082, 1148, 1278, 1287, 246, 1954]
